In [1]:
import pandas as pd
import geopandas as gpd
import os
from shapely.geometry import Point, Polygon

/Users/tjark/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/var/folders/93/smcvnv_s04v84f_dnqvd4xkw0000gn/T/ipykernel_17644/2429105346.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:/

In [4]:
pwd

'/Users/tjark/Documents/Python/CairoPopulation.nosync/tfc-git'

In [3]:
#move working directory up to acces data with relative paths
os.chdir("..")

## Read Greater Cairo Region addresses, buffer & disolve

In [5]:
# Load the point shapefile
address_points = gpd.read_file('data/raw/osm/addresses/GCR_addresses.gpkg')

In [6]:
# Convert the point geometry to a projected coordinate system
address_points = address_points.to_crs('EPSG:32636')

In [7]:
# Buffer the points with a 50m radius
buffered_address_points = address_points.buffer(50)

In [8]:
# Convert the buffered polygons back to a GeoDataFrame
buffered_address_points = gpd.GeoDataFrame(geometry=buffered_address_points, crs='EPSG:32636')

In [9]:
# Dissolve the buffered polygons
address_points_area = buffered_address_points.dissolve()

## Read Google open building centre points, filter by confidence interval, verify if inside previous polygon, merge

In [10]:
# Load the polygon data from the CSV file
building_points = gpd.read_file('data/raw/open-buildings/open_buildings_centroids.shp')

In [11]:
building_points.head()

,field_1,latitude,longitude,area_in_me,confidence,full_plus_,geometry
0,1,30.015059,31.427508,196.8340,0.7601,8G2H2C8H+22C7,POINT (31.42751 30.01506)
1,2,30.166752,31.316216,114.5702,0.7637,8G2H5888+PF45,POINT (31.31622 30.16675)
2,3,30.034948,31.189803,274.7086,0.6305,8G2H25MQ+XWFW,POINT (31.18980 30.03495)
3,4,30.129182,31.791867,465.7075,0.6922,8G2H4QHR+MPH8,POINT (31.79187 30.12918)
4,5,30.264915,31.262874,83.7047,0.6138,8G2H7777+X49Q,POINT (31.26287 30.26492)


In [12]:
# Delete buildings smaller than 40 squaremeters and confidence unter 0.7
building_points = building_points[building_points['area_in_me'] >= 40]
building_points = building_points[building_points['confidence'] >= 0.70]

In [13]:
# Check if the center points are within the dissolved polygon
center_points_within = building_points.within(address_points_area.unary_union)

In [14]:
# Delete center points within the dissolved polygon
filtered_center_points = building_points[~center_points_within]

In [15]:
filtered_center_points.head()

,field_1,latitude,longitude,area_in_me,confidence,full_plus_,geometry
0,1,30.015059,31.427508,196.8340,0.7601,8G2H2C8H+22C7,POINT (31.42751 30.01506)
1,2,30.166752,31.316216,114.5702,0.7637,8G2H5888+PF45,POINT (31.31622 30.16675)
5,6,29.963043,30.965676,104.7066,0.8485,7GXGXX78+677P,POINT (30.96568 29.96304)
6,7,30.355839,30.871322,186.6883,0.7822,8G2G9V4C+8GPF,POINT (30.87132 30.35584)
7,8,30.128563,31.288532,80.0793,0.8252,8G2H47HQ+CCFC,POINT (31.28853 30.12856)


In [16]:
filtered_center_points = filtered_center_points.to_crs('EPSG:32636')

In [17]:
joined_points = pd.concat([address_points, filtered_center_points])

In [18]:
joined_points.head()

,osm_id,addr:street,name,building,addr:housenumber,geometry,field_1,latitude,longitude,area_in_me,confidence,full_plus_
0,27565120.0,NaN,الجيزة,NaN,NaN,POINT (327496.394 3318698.896),NaN,NaN,NaN,NaN,NaN,NaN
1,31353319.0,NaN,بنها,NaN,NaN,POINT (325660.622 3371436.563),NaN,NaN,NaN,NaN,NaN,NaN
2,32538015.0,NaN,ميدان الجامع,NaN,NaN,POINT (338887.930 3330654.143),NaN,NaN,NaN,NaN,NaN,NaN
3,33471021.0,NaN,شل,NaN,NaN,POINT (343885.722 3328620.551),NaN,NaN,NaN,NaN,NaN,NaN
4,34712107.0,NaN,Misr Petrol,NaN,NaN,POINT (319399.108 3318793.667),NaN,NaN,NaN,NaN,NaN,NaN


## Keep only points within defined GCR

In [19]:
# Load the point shapefile
gcr_boundary = gpd.read_file('data/raw/eg_admin_boundaries/tfc_gcr_bounds.geojson')

In [20]:
gcr_boundary = gcr_boundary.to_crs('EPSG:32636')

In [21]:
# Check if the joined points are within the gcr bounds polygon
joined_points_within = joined_points.within(gcr_boundary.unary_union)

In [22]:
joined_points_within.head()

0     True
1    False
2     True
3     True
4     True
dtype: bool

In [23]:
# Delete joined points outside the gcr bounds polygon
gcr_addresses = joined_points[joined_points_within]

In [24]:
# Save the joined points to a new shapefile
gcr_addresses.to_file('data/interim/gcr_addresses.shp')

/var/folders/93/smcvnv_s04v84f_dnqvd4xkw0000gn/T/ipykernel_17644/1137287895.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gcr_addresses.to_file('data/interim/gcr_addresses.shp')
